<a href="https://colab.research.google.com/github/puneetgunwani/Adaptive-response-ai-model/blob/main/Moderate_Ltd_(India_%E2%80%93_UK)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install transformers torch pandas nltk spacy openai


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Emotion_classify_Data.csv to Emotion_classify_Data.csv


In [ ]:

import os
print(os.listdir())

['.config', 'results', 'Emotion_classify_Data.csv', 'logs', 'sample_data']


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('Emotion_classify_Data.csv')

# Display the first few rows to inspect the data
print(df.head())


                                             Comment Emotion
0  i seriously hate one subject to death but now ...    fear
1                 im so full of life i feel appalled   anger
2  i sit here to write i start to dig out my feel...    fear
3  ive been really angry with r and i feel like a...     joy
4  i feel suspicious if there is no one outside l...    fear


In [ ]:

# Check for missing values
print(df.isnull().sum())

# Remove rows with missing values (if any)
df = df.dropna()

Comment    0
Emotion    0
dtype: int64


In [ ]:
import re

# Function to clean text
def clean_text(text):
    # Remove punctuation, numbers, and convert text to lowercase
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    text = re.sub(r'\d+', '', text)      # Remove digits
    text = text.lower()                  # Convert to lowercase
    return text

# Apply the clean_text function to the 'Comment' column
df['cleaned_comment'] = df['Comment'].apply(clean_text)

# Display the cleaned data
print(df[['Comment', 'cleaned_comment']].head())

                                             Comment  \
0  i seriously hate one subject to death but now ...   
1                 im so full of life i feel appalled   
2  i sit here to write i start to dig out my feel...   
3  ive been really angry with r and i feel like a...   
4  i feel suspicious if there is no one outside l...   

                                     cleaned_comment  
0  i seriously hate one subject to death but now ...  
1                 im so full of life i feel appalled  
2  i sit here to write i start to dig out my feel...  
3  ive been really angry with r and i feel like a...  
4  i feel suspicious if there is no one outside l...  


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize the label encoder
label_encoder = LabelEncoder()

# Encode the emotion labels
df['emotion_label'] = label_encoder.fit_transform(df['Emotion'])

# Display the encoded labels
print(df[['Emotion', 'emotion_label']].head())

  Emotion  emotion_label
0    fear              1
1   anger              0
2    fear              1
3     joy              2
4    fear              1


In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and target (y)
X = df['cleaned_comment']  # Features: The cleaned comments (text)
y = df['emotion_label']    # Target: The corresponding emotions

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Check the size of training and test sets
print(len(X_train), len(X_test))


4749 1188


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Transform the test data (without fitting again)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Check the shape of the transformed data
print(X_train_tfidf.shape, X_test_tfidf.shape)

(4749, 5000) (1188, 5000)


In [ ]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Initialize and train the Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model's performance
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92       392
           1       0.95      0.90      0.93       416
           2       0.90      0.94      0.92       380

    accuracy                           0.92      1188
   macro avg       0.92      0.92      0.92      1188
weighted avg       0.92      0.92      0.92      1188



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10], 'max_iter': [100, 200, 500]}
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)
print(grid_search.best_params_)


{'C': 10, 'max_iter': 100}


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Initialize and train the Logistic Regression model with hyperparameters
model = LogisticRegression(C=10, max_iter=100)
model.fit(X_train_tfidf, y_train)

# Predict on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model's performance
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.93      0.95      0.94       392
           1       0.96      0.93      0.94       416
           2       0.94      0.95      0.95       380

    accuracy                           0.94      1188
   macro avg       0.94      0.94      0.94      1188
weighted avg       0.94      0.94      0.94      1188



In [ ]:
# Define responses for each emotion
emotion_responses = {
    'fear': 'I’m here for you. It’s okay to feel scared sometimes.',
    'anger': 'I understand your frustration. Let’s try to talk it out.',
    'joy': 'That’s wonderful to hear! Keep the positivity flowing!',
    'sadness': 'I’m really sorry you’re feeling this way. I’m here to help.',
    # Add more emotions and responses as needed
}

# Function to generate a response based on emotion
def generate_response(emotion):
    return emotion_responses.get(emotion, "I’m here to listen.")

# Predict emotion and generate response for the first test sample
predicted_emotion = label_encoder.inverse_transform([y_pred[0]])  # Get the predicted emotion
response = generate_response(predicted_emotion[0])

# Print the response
print("Predicted Emotion:", predicted_emotion[0])
print("Response:", response)


Predicted Emotion: anger
Response: I understand your frustration. Let’s try to talk it out.


In [ ]:

def chat_with_bot(user_input):
    # Clean the user input
    cleaned_input = clean_text(user_input)

    # Transform the input using TF-IDF
    user_input_tfidf = tfidf_vectorizer.transform([cleaned_input])

    # Predict the emotion
    predicted_label = model.predict(user_input_tfidf)
    predicted_emotion = label_encoder.inverse_transform(predicted_label)

    # Generate the response
    response = generate_response(predicted_emotion[0])

    return response

# Example usage:
user_input = "i sit here to write i start to dig out my feel"
print("Bot:", chat_with_bot(user_input))



Bot: I understand your frustration. Let’s try to talk it out.
